# **Deep Learning and CNN for Computer Vision, Hokkaido University**

## Day 2, Notebook -1: Image CLassification using CNN


In this session you will be implementing Convolutional Neural Network for Fashion MNIST dataset classification.

So lets get started!

## Tasks for this notebook:

1. Implementation of a CNN for Dogs and Cats classification using Keras API.
2. Train and test model


### Step 1: Import required packages

We will need tensorflow, numpy, os and keras


In [ ]:
import tensorflow as tf
import os
import numpy as np
import math, numpy as np
import sklearn.datasets
import matplotlib.pyplot as plt
import h5py
import glob
import cv2
import keras.utils as image
from tensorflow import keras

### Step 2: Download the Fashion Mnist dataset using keras

In [ ]:
fashionMnist=tf.keras.datasets.fashion_mnist

In [ ]:
# Load data from fashion mnist dataset using the load_data() method.
(train_images, train_labels), (test_images, test_labels) = fashionMnist.load_data()

In [ ]:
# Display the shapes of the training images
print(train_images.shape)
print(train_images.dtype)

In [ ]:
#define the class names for the fashion mnist dataset
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
## Display an image from the dataset
import matplotlib.pyplot as plt
plt.imshow(train_images[3])
print(train_labels[3])

**Note :** Scikit-learn import the Fashion MNIST dataset as a 1-D array while Keras API load the dataset in 28X28 format.

### Step 3: Normalize the dataset and split a small part of the training set into validation set


- Validation set: first 5000 samples (total 5000 samples)
- Training set: 5000 to remaining (total 55000 samples)

In [ ]:
## Using slicing to split the training to train and validation

train_images=train_images.reshape(60000, 28, 28, 1)
valid_images= train_images[:5000] / 255.0
valid_labels = train_labels[:5000]

train_images  = train_images[5000:] / 255.0
train_labels=train_labels[5000:]

test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images / 255.0


In [ ]:
# Print the shapes for Train, Validation, and Test dataset.
print(np.shape(train_images))
print(np.shape(valid_images))
print(np.shape(test_images))

[**Expected** Output]

(55000, 28, 28)
(5000, 28, 28)
(10000, 28, 28)

### Step 4:  Design the CNN Architecture

Design the following CNN architecture:

<img src='http://drive.google.com/uc?export=view&id=1KBmj460idGx6mWbAKsH1bsEYmjpthdPB' alt='Conv'>


Input: $64 X 64 X 3$ image

Activation function in CONV layer: Relu

Activation function in Output layer : softmax, 10 classes

**Hint:** Use Conv2D(), MaxPooling2D(), Flatten(), and Dense()



In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

## **Notes:**
* **Sequential model.** This is the simplest kind of Keras model, for neural networks which defines a SEQUENCE of layers.

* **Flatten.** Flatten just takes that image and turns it into a 1-dimensional vector.

* Next we add a second Dense hidden layer with 128 neurons, also using the ReLU activation function.  **Dense.** Add a layer to the neural network which is followed by activation function of ReLU. The ReLU only passes the value greater than 0 and for all other values of X it passes 0.
e.g. If X>0 return X, else return 0"

* Finally, we add a Dense output layer with 10 neurons (one per class), using the softmax activation function.

* **Softmax** The softmax takes a set of values and select the biggest one from the set of values.

## Step 5: Training the model

**"sparse_categorical_crossentropy":**   The dataset contains sparse labels and the classes are exclusive.

**One-hot vector encoding** This is sometime used for encoding the labels if there one target  probability per class for each instance. For example.
[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.] represent one-hot encoding for class 4. In such case, **"categorical_crossentropy"** loss is used.

**"sigmoid_crossentropy"** This loss is used for binary class classification problems and also **"sigmoid"** activation function is used instead of Softmax.



In [ ]:
# Using Plot_Model from Keras.Utils
model_img_file = 'CNN-model1.png'
tf.keras.utils.plot_model(model, to_file=model_img_file,
                          show_shapes=True,
                          show_layer_activations=False,
                          show_dtype=False,
                          show_layer_names=False )

# Also Try:
# show_shapes=True, show_layer_activations=True, show_dtype=True, show_layer_names=True

In [ ]:
# Compile the model and start training the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(train_images, train_labels, epochs=5)

In [ ]:
# Process the test images and find the accuracy
test_loss = model.evaluate(test_images, test_labels)

### Summary of the model

In [ ]:
model.summary()

## Step 6: Evaluation on test dataset

In [ ]:
model.evaluate(test_images, test_labels)

## Task: Image classification using Cats and Dogs Dataset.

###  Step: 1 Mount the Google Drive to access the Cats and Dogs Dataset
Reference: https://github.com/ardamavi/Dog-Cat-Classifier



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
cd /content/gdrive/MyDrive/Hokkaido Uni DL/Day-2

In [ ]:
!unzip Cats-Dogs-dataset-64.zip
!ls

### Step : 2 Image Generators: (Preparing the dataset for train, validation and testing)

In Keras  **keras.preprocessing.image.ImageDataGenerator** class  can be used to read images and extract labels from them via .flow_from_directory. The image generator can also be used for data augmentation. The image generators can used easily with Keras model that accept data generators as inputs. such as fit_generator, evaluate_generator, and predict_generator.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)


# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/gdrive/MyDrive/Hokkaido Uni DL/Day-2',  # This is the source directory for training images
        target_size=(64, 64),  # All images will be resized to 64X64
        batch_size=30,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow training images in batches of 32 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/content/gdrive/MyDrive/Hokkaido Uni DL/Day-2',  # This is the source directory for training images
        target_size=(64, 64),  # All images will be resized to 64X64
        batch_size=30,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

### Step 3: Create the CNN model:

Create the following CNN model:

<img src='http://drive.google.com/uc?export=view&id=1EAWFwp7T92q3Lm1ZrX9A2-wnvhfAfzSF' alt='Conv'>

Input: $64 X 64 X 3$ image

Activation function in CONV layer: Relu

Activation function in Output layer : sigmoid, 2 classes

**Hint:** Use Conv2D(), MaxPooling2D(), Flatten(), and Dense()

In [ ]:
## WRITE YOUR CODE HERE ## (~11 lines)
model1 = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(??),
    tf.keras.layers.MaxPooling2D(??),
    # The second convolution

    # The third convolution

    # The fourth convolution

    # Flatten the results to feed into a DNN

    # 512 neuron hidden layer

    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')

    ])

## END YOUR CODE HERE ##

In [ ]:
##Print the model summary
model1.summary()

In [ ]:
# Using Plot_Model from Keras.Utils
model_img_file = 'CNN-model2.png'
tf.keras.utils.plot_model(model1, to_file=model_img_file,
                          show_shapes=True,
                          show_layer_activations=False,
                          show_dtype=False,
                          show_layer_names=False )

# Also Try:
# show_shapes=True, show_layer_activations=True, show_dtype=True, show_layer_names=True

In [ ]:
## Compile the model and add loss, optimizer and metrics
## WRITE YOUR CODE HERE ## (~1 line)
model1.compile(loss=??,
              optimizer=tf.optimizers.Adam(),
              metrics=??)


In [ ]:
# Train/fit the model using the training and validation set.
history = model1.fit(
      train_generator,
      steps_per_epoch=8,
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

In [ ]:
## Plot the Training and Validation loss
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()



### Clearing all the resources

Terminate the kernel and free memory resources

In [ ]:
#import os, signal
#os.kill(os.getpid(), signal.SIGKILL)